<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_1_Data_Availability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prerequisites**

Before taking this module, you must have the following

* A Google Earth Engine Account
* A Google Cloud Project tied to your Google Earth Engine account.
* A Google Drive account.
* Google Colaboratory installed on your Google Drive

# **Introduction**

In this module, we will determine the availability of cloud-free optical data for a flood event of your interest. We are searching for a flood event in which there is a

- Sentinel-1 overpass over the area of interest during the time of interest
- Harmonized Landsat and Sentinel-2 (HLS) overpass over the area of interest during the time of interest
- low prevalence of cloud in all optical sensors used to create flood products (HLS, MODIS, VIIRS).

If we find a suitable flood event, we will export the area within the area of interest in which there is an overlap between the HLS and Sentinel-1 overpass geometries. Please note that this script is written with the intention of intercomparing floods where we have both a Sentinel-1 and an HLS overpass.

## **Step 1: Create GEE Directory**

- 1. Open the [Google Earth Engine Code Editor](https://code.earthengine.google.com/)

- 2. Navigate to the Assets Tab in the upper right.

- 3. Create a new folder by clicking the "New Button". Enter in a name or path for the folder. **This will be our parent directory, and will be where we upload all of the flood products**. My parent folder is called "users/mickymags/flood_intercomparison", but you can call yours whatever you like.

# Step 2: Define Inputs

After modifying the cells below to suit your case study and Earth Engine account, the rest of the cells should run as is.

To define our area of interest, we will type in the country name and either a list of provinces our a list of counties. The country name and province/county names must match the spelling as they appear in the [Global Administrative Unit Layers dataset](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_SIMPLIFIED_500m_2015_level2)

In [ ]:
# Google Earth Engine Variables
parent_directory = "users/mickymags/cambodia_test/"                                # Define the path to the directory you created in step 1. This will either begin with "users/..." or "projects/...""
aoi_asset_name = 'aoi'                                                             # Define what you want the asset name of your area of interest to be
gee_project_name = "servir-sco-assets"                                             # Type in the name of your Google Earth Engine Project.

# Case Study Variables
doi = "2024-10-01"                                                                 # Enter the Date of Interest in "YYYY-MM-DD"
country = 'Cambodia'                                                               # Enter country name as a string
province_list = 'Battambang,Kampong Cham,Kampong Chhnang,Kampong Thom,Pursat'      # Leave this as an empty string if using district_list
district_list = ''                                                                 # Leave this as an empty string if using province_list
flood_event_desc = 'Cambodia_test'                                                 # Enter in a description of your flood event

# Step 3: Import packages

We need to import a couple packages for this analysis.

1. The Earth Engine package will allow us to obtain data from Earth Engine and perform some operations on the data.
2. The [geemap](https://geemap.org/) Python package allows for further analysis and visualization using Earth Engine data.
3. The Google Drive package will allow us to export imagery and data from this notebook to your Google Drive.


In [ ]:
from google.colab import drive       # Import Google Drive
import ee                            # Import the Earth Engine package
import geemap                        # Import the Google Earth Engine Python package (geemap)
import os

Now we need to Authenticate Earth Engine, and initilaze it. After running the following cell, a popup window will appear. Follow the instructions therein to complete the initialization of Github.

In [ ]:
ee.Authenticate()
ee.Initialize(project = gee_project_name)

# Step 4: Get input data


For this project, we want to compare six flood products/software to each other. Some of these have different input datasets from optical and SAR sensors, which we detail below. In this notebook, we will examine the quality assurance infromation from all of the input datasets to determine if a case study is suitable for optical analysis.


* GFM: The Copernicus Global Flood Monitor. For more information see [this website](https://global-flood.emergency.copernicus.eu/technical-information/glofas-gfm/).
  * Input Dataset: Sentinel-1 uses IW GRDH product, which we can obtain from Google Earth Engine
*  TERRA: The Terra product used as input to the MCDWD product has the following DOI (http://doi.org/10.5067/MODIS/MOD09.NRT.061) (Slayback et al 2024). This product is not available in GEE. Thus, we will use a similar MODIS product in order to estimate the cloud cover in the region of interest
* AQUA: The Aqua product used as input to the MCDWD product is not specified. Email sent 09/05/24
* HLS: The HLS product is split into two datasets, one for Sentinel and one for Landsat. The Sentinel HLS product is not currently available on GEE.



In [ ]:
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")      # Sentinel-1 Ground Range Detected
hls_landsat = ee.ImageCollection("NASA/HLS/HLSL30/v002")
s2 = ee.ImageCollection("NASA/HLS/HLSS30/v002")          #s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
terra = ee.ImageCollection("MODIS/061/MOD09GA")
aqua = ee.ImageCollection("MODIS/061/MYD09GA")
viirs = ee.ImageCollection("NASA/VIIRS/002/VNP09GA")

Let's filter the Sentinel-1 collection to just those images that are used by the GFM flood Product. GFM ingests observations from the Sentinel-1 satellites that are acquired in the Interferometric Wide-swath (IW imaging mode) (Matgen et al 2022). GFM uses the Ground Range Detected at High Resolution (GRDH) product from Sentinel-1 to create the GFM Water Product (Matgen et al 2021).

In [ ]:
# Sentinel-1 Filtered to interferometric width
s1filt = sentinel1.filter(ee.Filter.eq('instrumentMode', 'IW'))\
                  .filter(ee.Filter.eq('resolution', 'H'))          # Filter to High Resolution Scenes

Get the day after our day of interest so we can filter our image collections to coincide with our date of interest.

In [ ]:
dayafter = ee.Date(doi).advance(1, "day")

Get an area of interest

In [ ]:
gaul = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

In [ ]:
def region_merger_v1(my_country, my_provinces):
  area_of_interest = ee.FeatureCollection([])

  country = gaul.filter(ee.Filter.eq("ADM0_NAME", my_country))

  splitstring = my_provinces.split(",")
  feature_list = []
  for y in splitstring:
    feature = country.filter(ee.Filter.eq("ADM1_NAME", y))
    feature_list.append(feature)
  for x in feature_list:
    area_of_interest = area_of_interest.merge(x)

  return area_of_interest

In [ ]:
def region_merger_v2(my_country, my_state, my_district_string):
  area_of_interest = ee.FeatureCollection([])

  country = gaul.filter(ee.Filter.eq("ADM0_NAME", my_country))
  state = country.filter(ee.Filter.eq("ADM1_NAME", my_state))

  splitstring = my_district_string.split(",")
  feature_list = []
  for y in splitstring:
    feature = state.filter(ee.Filter.eq("ADM2_NAME", y))
    feature_list.append(feature)
  for x in feature_list:
    area_of_interest = area_of_interest.merge(x)

  return area_of_interest

In [ ]:
if province_list == '':
  aoi = region_merger_v2(country, province_list, district_list)
else:
  aoi = region_merger_v1(country, province_list)

Let's print the aoi to the console to make sure that it contains our provinces. If you see "Feature Collection (0 elements...)", that means something went wrong. Check the spelling of your country, district, and provinces.

If you see "FeatureCollection (x elements...)" where x > 0, the code is working as intended. You can click the dropdown arrow to expand the collection and make sure it has all your provinces.

In [ ]:
aoi

In [ ]:
# Get the coordinates of the center of the AOI for mapping purposes
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

In [ ]:
s1final = s1filt.filterDate(doi, dayafter).filterBounds(aoi)

In [ ]:
s1vp = {
    'bands': ['VV'],
    'min': -25,
    'max': 0
}

In [ ]:
s1geom = s1final.geometry()

In [ ]:
# Merge the HLS Sentinel and HLS Landsat Image Collections
hls = s2.merge(hls_landsat)

# Filter the HLS collection to the day and region of interest
hls_final = hls.filterDate(doi, dayafter).filterBounds(aoi)

# Get the geometry of the above collection
hls_geom = hls_final.geometry()

Now we will define aoifinal, the final area of interest. this is the intersection of where our HLS overpass and our Sentinel-1 overpass is.

In [ ]:
aoifinal = hls_geom.intersection(s1geom)

let's print the intersected area below to make sure it has multiple vertices.

In [ ]:
aoifinal

In [ ]:
s1_mosaic = s1final.mosaic()

In [ ]:
s1vv_vp = {
    'bands': ['VV', 'VV', 'VV'],
    'min': - 25,
    'max': 0
}

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 5)
Map.addLayer(aoi, {}, 'region of interest')
Map.addLayer(s1geom, {}, 'Sentinel-1 Geometry')
Map.addLayer(hls_geom, {}, 'HLS Geometry')
Map.addLayer(aoifinal, {}, 'Final Area of Interest -- intersection of HLS and S1')
Map.addLayer(s1_mosaic, s1vv_vp, 'Sentinel1 VV')

Map.addLayerControl()
Map

In [ ]:
# Create a bounding box around the area of interest
aoibb = aoifinal.bounds()

Let's mount google drive. This will allow us to export our area of interest to Google Drive. The next cell will kick off another pop-up. Click through the pop-up to connect to Google Drive.

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
os.mkdir('Flood_Intercomparison')

In [ ]:
cd ..

In [ ]:
cd drive/MyDrive/Flood_Intercomparison

In [ ]:
aoifc = ee.FeatureCollection(aoifinal)
aoibb_fc = ee.FeatureCollection(aoibb)

In [ ]:
geemap.ee_export_vector_to_drive(aoifc, description='drive_export_aoi'+flood_event_desc,folder='Flood_Intercomparison',fileFormat='GeoJSON')
geemap.ee_export_vector_to_drive(aoibb_fc, description='drive_export_aoi_bbox'+flood_event_desc,folder='Flood_Intercomparison',fileFormat='GeoJSON')

geemap.ee_export_vector_to_asset(aoifc, description='exporting_final_aoi'+flood_event_desc, assetId= parent_directory + 'aoi')
geemap.ee_export_vector_to_asset(aoibb_fc, description='exporting_final_aoi_bbox'+flood_event_desc, assetId= parent_directory + 'aoi_bbox')

The area of interest and its bounding box will now be available in Google Drive, under the Flood_Intercomparison folder. They will also be available in Google Earth Engine, under the folder you defined in Step 1. You can now move to Module 2.

# References


- Matgen, Patrick. "Provision of an Automated, Global, Satellite-based Flood Monitoring Product for the Copernicus Emergency Management Service. "GFM D6 Product Definition Document. Issue 1 Version 1.4. April 20, 2022.

- Slayback, Dan. MODIS NRT Global Floodo Product User Guide. Revision D. NASA LANCE, 16 April 2024. https://www.earthdata.nasa.gov/s3fs-public/2024-04/MCDWD_UserGuide_RevD.pdf

- Roger, J.C. et al. "Suomi-NPP VIIRS Surface Reflectance User's Guide". NASA Land SIPS. Version 2.0. April 2023. https://lpdaac.usgs.gov/documents/1657/VNP09_User_Guide_V2.pdf
